<a href="https://colab.research.google.com/github/mahdi943/two_capital_furthest_away/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Using Capitals.txt dataset;
Find two capital cities furthest away from each other and the distance between them.

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

In the first step, we need removing double space characters as well as changing the ',' character in the float numbers to '.' in Capitals.txt

In [ ]:
myFile = open('/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Cc.txt', 'w')
with open('/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Capitals.txt', 'r') as f:
  for line in f:
    st = line.replace(' ', '').replace(',','.')
    myFile.write(st)
f.close()
myFile.close()

I did small change from Q-1 to sort from larg to small for distance. The result is two capital from Paraguay(Asunción) and Taiwan(Taipei).

In [ ]:
from pyspark import SparkContext
import math
sc = SparkContext.getOrCreate()

capRDD = sc.textFile("/content/drive/MyDrive/Colab-Notebooks/Big-data/HW1/Dataset/Cc.txt")

def saveCoord(line):
    rec = line.split("\t")
    name = rec[2]
    lat = float(rec[3])
    lng = float(rec[4])
    return(name,lat,lng)

capRDD1 = capRDD.map(lambda x : saveCoord(x) )
capRDD2 = capRDD.map(lambda x : saveCoord(x) )

capRDD3 = capRDD1.cartesian(capRDD2)
capRDD3 = capRDD3.map(lambda x : x[0]+x[1])



def distance(city1, city2, lat1, lon1, lat2, lon2):
  if city1 != city2:
    radius = 6371  # km
    lat1 = float(lat1)
    lon1 = float(lon1)
    lat2 = float(lat2)
    lon2 = float(lon2)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
      math.cos(math.radians(lat1)) *
      math.cos(math.radians(lat2)) * math.sin(dlon / 2) *
      math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return (city1,city2,d)


capRDD3 = capRDD3.map(lambda x : distance(x[0],x[3],x[1],x[2],x[4],x[5]))
#print(capRDD3.take(10))

# since we applide cartesian, we should apply filter for both 
capRDD3  = capRDD3.filter(lambda x: x is not None)
capRDD3 = capRDD3.sortBy(lambda x : x[2],ascending=False)

print("Two furthest capital cities in the world:",capRDD3.take(1))

Two furthest capital cities in the world: [('Asunción', 'Taipei', 19927.05728016978)]
